The manual: http://data.atoc.org/sites/all/themes/atoc/files/RSPS5045.pdf

The data: http://data.atoc.org/fares-data

In [1]:
import pandas as pd

### Flow file

In [2]:
out = pd.read_table("../data/RJFAF575/RJFAF575.FFL")

In [3]:
#out_truncated = 
out[5:].head()

,/!! Start of file
5,RF0027003201000000AS3112299902012020ATO01Y0000002
6,RF0027003301000000AS3112299902012020ATO01Y0000034
7,RF0027003401000000AS3112299902012020ATO01Y0000121
8,RF0027003501000000AS3112299902012020ATO01Y0000174
9,RF0027006401000000AS3112299902012020ATO01Y0000049


In [53]:
flow_data = out.copy()

In [54]:
flow_data.rename(columns={"/!! Start of file                                                               ":"records"}, inplace=True)

In [55]:
# this dictinobary contains information for what type of ticket there is. 
dict_flow_records = {
    "update_marker": (0, 1),
    "record_type": (1, 2),
    "origin_code": (2, 6),
    "destination_code": (6, 10),
    "route_code": (10, 15),
    "status_code": (15, 18), 
    "usage_code": (18, 19),
    "direction": (19, 20),
    "end_date": (20, 28),
    "start_date": (28, 36),
    "toc": (36, 39), 
    "cross_london_ind": (39, 40),
    "non_standard_disc_ind": (40, 41),
    "publication_ind": (41, 42),
    "flow_id": (42, 49)
}

In [56]:
# this dictionary holds the information for for what the fares are for the different types
# it joins to the previous one by the flow_id
dict_fare_records = {
    "update_marker": (0, 1),
    "record_type": (1, 2),
    "flow_id": (2, 9),
    "ticket_code": (9, 12),
    "fare": (12, 20), # in pence
    "restriction_code": (20, 22)
}

In [57]:
lengths_of_records = flow_data["records"].str.len()

In [58]:
flow_records = flow_data[lengths_of_records == 49].copy()

for key, pair_indices in dict_flow_records.items():
    flow_records[key] = flow_records["records"].str[pair_indices[0]:pair_indices[1]]
    
fare_records = flow_data[lengths_of_records == 22].copy()

for key, pair_indices in dict_fare_records.items():
    fare_records[key] = fare_records["records"].str[pair_indices[0]:pair_indices[1]]

In [59]:
flow_data.tail()

,records
5938765,RT07364537DS00005080
5938766,RT0736453CDR00000940B1
5938767,RT0736453SDR00001130
5938768,RT0736453SDS00000850
5938769,/!! End of file (18/05/2020)


In [60]:
merged_data = flow_records.merge(
    fare_records,
    on=["flow_id", "update_marker"])

In [49]:
sum(merged_data["origin_code_x"] == merged_data["origin_code_y"]) / len(merged_data)

0.0

In [61]:
merged_data.head()

,records_x,update_marker,record_type_x,origin_code,destination_code,route_code,status_code,usage_code,direction,end_date,...,toc,cross_london_ind,non_standard_disc_ind,publication_ind,flow_id,records_y,record_type_y,ticket_code,fare,restriction_code
0,RF0027003201000000AS3112299902012020ATO01Y0000002,R,F,0027,0032,01000,000,A,S,31122999,...,ATO,0,1,Y,0000002,RT00000027TF00005420,T,7TF,00005420,
1,RF0027003201000000AS3112299902012020ATO01Y0000002,R,F,0027,0032,01000,000,A,S,31122999,...,ATO,0,1,Y,0000002,RT00000027TS00003610,T,7TS,00003610,
2,RF0027003301000000AS3112299902012020ATO01Y0000034,R,F,0027,0033,01000,000,A,S,31122999,...,ATO,0,1,Y,0000034,RT00000347TF00006360,T,7TF,00006360,
3,RF0027003301000000AS3112299902012020ATO01Y0000034,R,F,0027,0033,01000,000,A,S,31122999,...,ATO,0,1,Y,0000034,RT00000347TS00004240,T,7TS,00004240,
4,RF0027003401000000AS3112299902012020ATO01Y0000121,R,F,0027,0034,01000,000,A,S,31122999,...,ATO,0,1,Y,0000121,RT00001217TF00007790,T,7TF,00007790,


## Locations file
Locations, including group locations

In [62]:
locations_file = pd.read_table("../data/RJFAF575/RJFAF575.LOC")

In [69]:
locations_file.rename(
    columns={'/!! Start of file                                                               ':"records"},
    inplace=True)

In [70]:
dictionary_locations = {
    "update_marker": (0, 1),
    "record_type": (1, 2),
    "uic_code": (2, 9),
    "end_date": (9, 17), 
    "start_date": (17, 25),
    "quote_date": (25, 33),
    "admin_area_code": (33, 36),
    "nlc_code": (36, 40),
    "description": (40, 56),
    "crs_code": (56, 59),
    "resv_code": (59, 64),
    "ers_country": (64, 66),
    "ers_code": (66, 69),
    "fare_group": (69, 75),
    "county": (75, 77),
    "pte_code": (77, 79),
    "zone_no": (79, 83),
    "zone_ind": (83, 85),
    "region": (85, 86),
    "hierarchy": (86, 87),
    "cc_desc_out": (87, 128),
    "cc_desc_rtn": (128, 144),
    "atb_desc_out": (144, 204),
    "atb_desc_rtn": (204, 234),
    "special_facilities": (234, 260),
    "lul_direction_ind": (260, 261),
    "lul_uts_mode": (261, 262),
    "lul_zone_1": (262, 263),
    "lul_zone_2": (263, 264),
    "lul_zone_3": (264, 265),
    "lul_zone_4": (265, 266),
    "lul_zone_5": (266, 267),
    "lul_zone_6": (267, 268),
    "lul_uts_london_stn": (268, 269),
    "uts_code": (269, 272),
    "uts_a_code": (272, 275),
    "uts_ptr_bias": (275, 276),
    "uts_offset": (276, 277),
    "uts_north": (277, 280),
    "uts_east": (280, 283),
    "uts_south": (283, 286),
    "uts_west": (286, 289)
}

In [71]:
dict_associated_stations = {
    "update_marker": (0, 1),
    "record_type": (1, 2), # should be A
    "uic_code": (2, 9),
    "end_date": (9, 17),
    "assoc_uic_code": (17, 24),
    "assoc_crs_code": (24, 27)
}

In [72]:
dict_railcard_geography = {
    "update_marker": (0, 1),
    "record_type": (1, 2),
    "uic_code": (2, 9),
    "railcard_code": (9, 12),
    "end_date": (12, 20)
}

In [73]:
dict_synonym = {
    "update_marker": (0, 1),
    "record_type": (1, 2), # "S"
    "uic_code": (2, 9),
    "end_date": (9, 17),
    "start_date": (17, 25),
    "description": (25, 41)
}

In [74]:
lengths_of_records_locations = locations_file["records"].str.len()

In [75]:
lengths_of_records_locations

0         80
1         80
2         80
3         80
4         80
          ..
425056    41
425057    41
425058    41
425059    41
425060    28
Name: records, Length: 425061, dtype: int64

In [24]:
locations_file.values[7]

array(['RG7000330200920191901201918012019LONDON ZONES 1-3     '],
      dtype=object)

### Data Feed